In [1]:
import numpy as np
import cv2
import urllib.request
import os
import csv
import tensorflow
import urllib.request

from clip_retrieval.clip_client import ClipClient, Modality
from matplotlib import pyplot as plt
from deepface import DeepFace
from pathlib import Path
from tqdm import tqdm

from matplotlib import pyplot as plt
import plotly.express as px
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
import torch
import math
import pandas

In [16]:
# Function used to load images from a specified folder
# can be used to load custom dataset or images already retrived via CLIP
def LoadImagesFromFolder(folder_path):
    """ 
    Description:
    This function is used to load images from the specified folder, mainly to avoid repeatedly using CLIP to retrieve the same images. 
    However it can be used to load other images (GenerativeImages).

    Parameters:
    folder_path (str) - Path to the folder in which the images are stored.

    Returns:
    image_list (list) - List containing the loaded images.
    """

    # Initialize an empty list to store the loaded images
    image_list = []

    list_of_files = os.listdir(folder_path)

    # Loop through the files in the folder
    for file_name in list_of_files:
        # Construct the full path to the image file
        image_path = os.path.join(folder_path, file_name)

        # Load the image using cv2 in BGR format
        image_bgr = cv2.imread(image_path)

        if image_bgr is not None:
            # Convert BGR to RGB format
            image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

            # Append the RGB image to the list
            image_list.append([file_name,image_rgb])

    return image_list

In [28]:
# Reading validation csv
csv_path = "C:\\Users\\User\\FYP\\FairFace\\val_labels.csv"
csvFile = pandas.read_csv(csv_path)

'val/1.jpg'

In [58]:
csvFile["race"].unique()

array(['East Asian', 'White', 'Latino_Hispanic', 'Southeast Asian', 'Black', 'Indian', 'Middle Eastern'], dtype=object)

In [17]:
# Loading images from the folder
folder_path = "C:\\Users\\User\\FYP\\FairFace\\val"
list_of_files = LoadImagesFromFolder(folder_path)

In [115]:
objs = DeepFace.analyze(list_of_files[0][1], actions = ['age', 'gender', 'race'], enforce_detection=False,  detector_backend="mtcnn", silent=True)
objs

1/1 [==============================] - 0s 73ms/step


[{'age': 32,
  'region': {'x': 17, 'y': 0, 'w': 165, 'h': 195},
  'gender': {'Woman': 5.582701042294502, 'Man': 94.41729784011841},
  'dominant_gender': 'Man',
  'race': {'asian': 48.62211040390196,
   'indian': 10.44402929806884,
   'black': 8.022934676583688,
   'white': 5.966442061450604,
   'middle eastern': 3.469080684726279,
   'latino hispanic': 23.475405482971993},
  'dominant_race': 'asian'}]

In [86]:
processed_images = []
for file_name, image in list_of_files:
    # emotion removed as FairFace does not have emotion labels
    objs = DeepFace.analyze(image, actions = ['age', 'gender', 'race'], enforce_detection=False,  detector_backend="mtcnn", silent=True)
    processed_images.append([file_name, image, objs])
    

1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 69ms/step


KeyboardInterrupt: 

In [116]:
gender_tp_count = 0
gender_incorrect_count = 0
age_tp_count = 0
age_incorrect_count = 0
race_tp_count = 0
race_incorrect_count = 0
error_count = 0

for file_name, image, objs in processed_images[:372]:
    index = list(csvFile["file"]).index("val/"+file_name)

    try:
        # Checking for correct age prediction
        range_val = csvFile["age"][index].split("-")
        if int(objs[0]["age"]) in range(int(range_val[0]), int(range_val[1])):
            age_tp_count += 1
        else:
            age_incorrect_count += 1

        # Checking for correct gender prediction   
        if csvFile["gender"][index].lower() == "male" and objs[0]["dominant_gender"].lower() == "man":
            gender_tp_count += 1
        elif csvFile["gender"][index].lower() == "female" and objs[0]["dominant_gender"].lower() == "woman":
            gender_tp_count += 1
        else:
            gender_incorrect_count += 1

        # Checking for correct race prediction
        if "asian" in csvFile["race"][index].lower() and objs[00]["dominant_race"].lower() == "asian":
            race_tp_count += 1
        elif csvFile["race"][index].lower() == objs[0]["dominant_race"].lower():
            race_tp_count += 1
        else:
            race_incorrect_count += 1
    except:
        error_count += 1
        pass

print(gender_tp_count)
print(race_tp_count)
print(age_tp_count)
print(error_count)

250
208
92
4


In [117]:
print("Gender Result:", gender_tp_count/(gender_tp_count+gender_incorrect_count-error_count)*100)
print("Race Result:", race_tp_count/(race_tp_count+race_incorrect_count-error_count)*100)
print("Age Result:", age_tp_count/(age_tp_count+age_incorrect_count-error_count)*100)

Gender Result: 68.68131868131869
Race Result: 57.14285714285714
Age Result: 25.274725274725274
